In [1]:
import networkx as nx;
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;
import random;
import itertools;
import numpy as np;

In [2]:
def MST_MFNIP (G, budget, network, gamma):
    
    mst_file = open('MST_MFNIP_'+network+'_b'+str(budget)+'.mst', "w");
    
    mst_file.write('# gamma values \n');
    for i,j in G.edges:
        mst_file.write('%s %g \n' %(gamma[i,j].VarName, gamma[i,j].x));
    mst_file.write('#\n');
    
    mst_file.close();

In [3]:
def MaxFlow (network, Gamma):
    
    networkCSV = network+'.csv';
    
    # Reading network file
    with open(networkCSV, newline='') as f:
        reader = csv.reader(f);
        row1 = next(reader);
        s = int(row1[0]);             # Source node
        t = int(row1[1]);             # Sink node
    
        G = nx.DiGraph();
        data = pd.read_csv(networkCSV, skiprows=1, header=None);
        n_edge = len(data.index+1);
    
        for i in range(n_edge): 
            G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);
    
    maxFlow = gp.Model("MaxFlow"); 
    
    x = maxFlow.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    
    maxFlow.addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) - 
                       gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
    
    for i,j in G.edges:
        maxFlow.addConstr(x[i,j] <= G.edges[i,j]['capacity']*(1-Gamma[i,j]));
            
    maxFlow.setObjective(x[t,s], GRB.MAXIMIZE);
    
    maxFlow.update();
    maxFlow.setParam("OutputFlag", 0);
    maxFlow.optimize();
    
    X = {};
    for i,j in G.edges:
        X[i,j] = x[i,j].x;
    
    return X;

In [4]:
# Max Problem (to obtain the values of x and z given an interdiction)

def InnerProblem (network, Gamma, M, s, t):
    
    networkCSV = network+'.csv';
    
    # Reading network file
    with open(networkCSV, newline='') as f:
        reader = csv.reader(f);
        row1 = next(reader);
        s = int(row1[0]);             # Source node
        t = int(row1[1]);             # Sink node
    
        G = nx.DiGraph();
        data = pd.read_csv(networkCSV, skiprows=1, header=None);
        n_edge = len(data.index+1);
    
        for i in range(n_edge): 
            G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);
    
  
    max_model = gp.Model("Max_z");
    
    x = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    z = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    alpha = max_model.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    theta = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    
    sum_z = max_model.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = M);
            
    max_model.addConstrs(gp.quicksum(x[j,i] for i in G.successors(j))
                         - gp.quicksum(x[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
            
    for i,j in G.edges:
        max_model.addConstr(x[i,j] <= (1-Gamma[i,j])*G.edges[i,j]['capacity']);
            
    for i,j in G.edges:
        max_model.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
            
    max_model.addConstr(alpha[t] - alpha[s] >= 1);
            
    max_model.addConstr(x[t,s] >= gp.quicksum((1-Gamma[i,j])*G.edges[i,j]['capacity']*theta[i,j]
                                                     for i,j in G.edges));
            
    for i,j in G.edges:
        if (G.edges[i,j]['special'] == 1):
            max_model.addConstr(x[i,j] >= (1/M)*z[i,j]);
            
    max_model.addConstr(sum_z <= gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges));
            
    max_model.setObjective(sum_z, GRB.MAXIMIZE); 
            
    max_model.setParam("IntegralityFocus",1);
    max_model.update();
    max_model.setParam("OutputFlag", 0);
    max_model.optimize();
    
    if max_model.SolCount == 0:
        print("No feasible solution available (Max-Problem)");
    
    
    X = {};
    Z = {};
    
    for i,j in G.edges:
        X[i,j] = x[i,j].x;
        Z[i,j] = z[i,j].x;
      
    return X, Z;

In [5]:
def HT_solver_MFNIP (network, budget, rate, T_Limit, summaryName):
    
    start_time = time.time(); 
    
    mod_network = 'Modified_'+network;
    
    networkCSV = network+'.csv';
    mod_networkCSV = 'Modified_'+network+'.csv';
    
    
    # Reading network file
    with open(mod_networkCSV, newline='') as f:
        reader = csv.reader(f);
        row1 = next(reader);
        s = int(row1[0]);             # Source node
        t = int(row1[1]);             # Sink node
    
        G = nx.DiGraph();
        data = pd.read_csv(mod_networkCSV, skiprows=1, header=None);
        n_edge = len(data.index+1);
    
        for i in range(n_edge): 
            G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], trafficker=data.iat[i,5], 
                    bottom=data.iat[i,6], victim=data.iat[i,7]);
    
    for i,j in G.edges:           # Applying rate to trafficker's capacity (HT Networks)
        if i == s:
            G.edges[i,j]['capacity'] = math.floor(rate*G.edges[i,j]['capacity']);
     
    A = 0;   # Number of special arcs
    T = 0;   # Number of Traffickers 
    B = 0;   # Number of intermediaries (Bottoms)
    V = 0;   # Number of Victims
    U = 0;   # Maximum capacity
    
    for i,j in G.edges:
        if G.edges[i,j]['special'] == 1:
            A = A + 1;
        if G.edges[i,j]['trafficker'] == 1:
            T = T + 1;
        elif G.edges[i,j]['bottom'] == 1:
            B = B + 1;
        elif G.edges[i,j]['victim'] == 1:
            V = V + 1;
            U = U + G.edges[i,j]['capacity'];
    
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ###

    
    ### Optimization Model ###
    
    M = A;
    
    #instance = 'LogFile_'+MFNIP_network+'_b'+str(budget)+'.txt';
    
    model = gp.Model("HT_MFNIP_method"); 
    
    gamma = model.addVars(G.edges, vtype=GRB.BINARY, name="gamma"); 
    x = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="x", lb = 0, ub = GRB.INFINITY);
    theta = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="theta", lb = 0, ub = 1); 
    alpha = model.addVars(G.nodes, vtype=GRB.CONTINUOUS, name="alpha", lb = 0, ub = 1); 
    beta = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="beta", lb = 0, ub = 1); 
    
    
    model.addConstr(gp.quicksum(G.edges[i,j]['cost']*gamma[i,j] for i,j in G.edges) <= budget);
    
    model.addConstr(x[t,s] >= gp.quicksum(G.edges[i,j]['capacity']*beta[i,j] for i,j in G.edges));
    
    model.addConstr(alpha[t] - alpha[s] >= 1);
    
    for i,j in G.edges:
        model.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
        model.addConstr(beta[i,j] + gamma[i,j] - theta[i,j] >= 0);
        model.addConstr(x[i,j] - G.edges[i,j]['capacity']*(1-gamma[i,j]) <= 0);
            
    model.addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) -
                     gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
    
    
    
    model.setObjective(x[t,s], GRB.MINIMIZE);
    
    model.setParam("IntegralityFocus",1);
    model.setParam('TimeLimit', T_Limit); 
    model.update();
    
    model.setParam("OutputFlag", 0);
    model.optimize();
  
    status = model.status;
    end_time = time.time();
    run_time = round(end_time - start_time, 2);
    
    now = datetime.now();
    
    obj = round(model.objVal);
    sol = A - obj;
    OptGap = round(model.MIPGap, 2);
    
    
    # ~~~~~~~~~~~ MAX FLOW ON THE ORIGINAL NET ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    Gamma = {};
    for i,j in G.edges:
        Gamma[i,j] = gamma[i,j].x;
    
    X, Z = InnerProblem (network, Gamma, M, s,t);
    
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    
    # ~~~~~~~~~~ CREATING MST (MIP START) FILE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    
    # MST_MFNIP (G, budget, network, gamma);
    
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    
    
    
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ####
    
    ### Print to the screen and to result file #######################
    
    file = open('HT_'+network+'_b'+str(budget)+'_r'+str(rate)+'.txt', "w");
    file.write('Instance: %s, budget %g, rate %g \n' %(network, budget, rate));
    file.write('Instance executed at: %s \n\n' %now.strftime("%c"));
    
    file.write('Number of Nodes: %g' % (T+B+V) +'\n');
    file.write('Number of Traffickers: %g' % T +'\n');
    file.write('Number of Bottoms: %g' % B +'\n');
    file.write('Number of victims: %g' % (V) +'\n');
    file.write('Budget: %g' % budget +'\n\n');
    
    
    print('\n')
    print('Instance: %s, budget %g, rate %g \n' %(network, budget, rate));
    if status == 2:
        print('Status: Optimal');
        print('Number of special arcs without flow: %g' % sol);
        print('Number of special arcs with flow: %g' % obj);
        
        file.write('Status: Optimal\n');
        file.write('Number of special arcs without flow: %g \n' % sol);
        file.write('Number of special arcs with flow: %g \n' % obj);
        
    elif status == 9:
        print('Status: Time Limit');
        print('Number of special arcs without flow: %g' % sol);
        print('Number of special arcs with flow: %g' % obj);     
        print('Optimality Gap: %g' % OptGap);
        
        file.write('Status: Time Limit \n');
        file.write('Number of special arcs without flow: %g \n' % sol);
        file.write('Number of special arcs with flow: %g \n' % obj);     
        file.write('Optimality Gap: %g \n' % OptGap);
              
    else:
        print('Status: %g' %status);
        file.write('Status: %g \n' %status);
    
    print('\nNumber of special arcs: %g' %A);
    print('Max-Flow (Original Network): %g \n' %X[t,s]);
    file.write('Max-Flow (Original Network): %g \n' %X[t,s]);
    
    print('run time: %g sec \n' %run_time);
    file.write('run time: %g sec \n' %run_time);
    
    print('Intediction:')
    for i,j in G.edges:
        if gamma[i,j].x >= 0.0001:
            print("arc (%g,%g), gamma = %g " %(i,j, gamma[i,j].x));
    print('\n');
    
    '''
    file.write('\nTrafficker Capacity (Modified Net): \n');
    for i,j in G.edges:
        if i==s:
            file.write('(%g, %g) : %g \n' %(i,j,G.edges[i,j]['capacity']));
    '''
    
    int_T = 0;
    int_B = 0;
    int_V = 0;
    file.write('\nInterdiction plan: \n');
    for i, j in G.edges: 
        if gamma[i,j].x >= 0.0001: 
            if G.edges[i,j]['trafficker'] == 1:
                file.write('Trafficker: %g, gamma = %g \n' %(j, gamma[i,j].x));
                int_T = int_T + 1;
            elif G.edges[i,j]['bottom'] == 1:
                file.write('Bottom: %g, gamma = %g \n' %(j, gamma[i,j].x));
                int_B = int_B + 1;
            elif G.edges[i,j]['victim'] == 1:
                file.write('Victim: %g, gamma = %g \n' %(i, gamma[i,j].x));
                int_V = int_V + 1;
    
    file.write('\nVictim nodes with flow \n');    
    for i, j in G.edges:
        if G.edges[i,j]['special'] == 1:
            if X[i,j] >= 0.0001:
                file.write('victim: %g, \t Flow: %g \n' %(i, X[i,j]));
    
    file.write('\n');
    file.close();
    
    rowFields = [network, budget, rate, T+B+V, G.number_of_edges(), T, B, V, round(obj), OptGap, X[t,s],
                 int_T, int_B, int_V, run_time];
    
    with open(summaryName, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile);
        csvwriter.writerow(rowFields);
        csvfile.close();


In [6]:
n_Networks = 10;
Budget = [0, 4, 8, 12, 16, 20, 24];
Rate = [0.5, 0.75, 1]

T_Limit = 7200;


summaryName = "HT_MFNIPmethod_Summary.csv"
file_summary = open(summaryName, "w");
file_summary.write('Instance,Budget,rate,Nodes,Arcs,Traffickers,Bottoms,Victims,ObjVal,');
file_summary.write('ObtGap,MaxFlow,Int_L1,Int_L2,Int_L3,Time\n');
file_summary.close();

for n in range(1, n_Networks+1):
    network = 'Network'+str(n);
    
    for budget in Budget:
        
        for rate in Rate:
            HT_solver_MFNIP (network, budget, rate, T_Limit, summaryName);

Set parameter TokenServer to value "license8.clemson.edu"
Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 7200
Set parameter IntegralityFocus to value 1


Instance: Network1, budget 0, rate 0.5 

Status: Optimal
Number of special arcs without flow: 0
Number of special arcs with flow: 36

Number of special arcs: 36
Max-Flow (Original Network): 195 

run time: 0.07 sec 

Intediction:


Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 7200
Set parameter IntegralityFocus to value 1


Instance: Network1, budget 0, rate 0.75 

Status: Optimal
Number of special arcs without flow: 0
Number of special arcs with flow: 36

Number of special arcs: 36
Max-Flow (Original Network): 195 

run time: 0.04 sec 

Intediction:


Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 7200
Set parameter IntegralityFocus to value 1


Instance: Network1, budget 0, rate 1 

Status: Optimal
Number of special arcs without flow: 0
Number of s